### Insights and Code References Attribution

This work incorporates insights and code snippets courtesy of Hugging Face, published on Hugging Face's blog. We extend our gratitude to the original authors and Hugging Face for sharing their valuable resources with the community.

Here are few resources used for reference.

https://huggingface.co/blog/llama2

https://github.com/huggingface/trl/blob/main/examples/scripts/sft.py

https://colab.research.google.com/drive/1BiQiw31DT7-cDp1-0ySXvvhzqomTdI-o?usp=sharing

https://huggingface.co/datasets/timdettmers/openassistant-guanaco/viewer/default/test?row=11

https://colab.research.google.com/drive/1VoYNfYDKcKRQRor98Zbf2-9VQTtGJ24k?usp=sharing


## Environment

We will run this code on google colab with T4 GPU

## Install required Libraries

trl - Transformer Reinforcement Learning

transformers - Transformers provides APIs and tools to easily download and train state-of-the-art pretrained models. 

accelerate - Accelerate  enables the same  code to be run across different distributed configurations.

peft -  Parameter Efficient Fine Tuning 

datasets - for  accessing and sharing data

bitsandbytes - For quantization 

wandb - Weights and Balances for logging, metrics tracking


In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarr

# Understanding of Basic Concepts

### peft - Parameter Efficient Fine Tuning

Parameter Efficient Fine-Tuning (PEFT) refers to a set of techniques in machine learning aimed at updating large pre-trained models in a way that requires altering only a small fraction of the model's parameters. This approach is particularly useful when working with very large models, such as those used in natural language processing (NLP) and computer vision, where full model retraining can be prohibitively expensive in terms of computational resources and time.

### LoRA 

In the context of parameter-efficient fine-tuning for machine learning models, LoRA and QLoRa refer to specific techniques designed to update large pre-trained models more efficiently.

LoRA (Low-Rank Adaptation):

Low-Rank Adaptation is a technique used to fine-tune large pre-trained models in a parameter-efficient manner. Instead of updating all the parameters of a model, LoRA focuses on adapting only a small set of parameters, typically by introducing and training low-rank matrices that modify the weights of specific layers (like the attention or feed-forward layers in a Transformer model). This approach allows for significant customization of the model to new tasks with minimal changes to the overall parameter set, preserving the original strengths of the pre-trained model while adapting it to specific needs.

Imagine you have a huge, complex book (representing a large pre-trained model) that you want to customize slightly without rewriting the whole thing. LoRA allows you to do just that by adding a few notes or annotations (small changes) to some pages, making the book more relevant to your specific needs without altering its original content significantly. In technical terms, LoRA introduces small, trainable adjustments to certain parts of the model while keeping the vast majority of the original parameters fixed. This makes the fine-tuning process much more efficient, as you're only tweaking a tiny fraction of the model.

![lora](lora.jpeg)

The image above is designed to visually explain the concept of Low-Rank Adaptation (LoRA) within Parameter Efficient Fine-Tuning (PEFT), with a focus on resembling a neural network. It illustrates how specific LoRA modules act as targeted enhancements to the neural network, allowing for precise and efficient adjustments to adapt the model for new tasks.


### QLoRa 

QLoRa (Quantized Low-Rank Adaptation):

Quantization, in the context of machine learning, involves reducing the precision of the numerical representations used in a model. For example, reducing floating-point numbers from 32-bit to 8-bit integers. This process helps decrease the model's memory footprint and can speed up inference, making the model more efficient, especially on hardware with limited computational resources or specific accelerators designed to work with lower-precision arithmetic.

Building on the previous analogy, if LoRA is like adding handwritten notes to the book, QLoRa is akin to using a shorthand or a more condensed notation for those notes. QLoRa further compresses the changes you make (the annotations or notes) by quantizing them, which means representing these adjustments with fewer bits. This not only makes the fine-tuning process more efficient but also reduces the memory and computational resources needed to apply these customizations.

![qlora](qlora.jpeg)

In very simple terms, both LoRA and QLoRa are about making smart, minimal updates to a large, pre-existing "knowledge base" (the pre-trained model) to adapt it for specific tasks. LoRA does this by adding small modifications, and QLoRa goes a step further by compressing these modifications to be even more resource-efficient.

### Hugging Face TRL and SFT

Transformer Reinforcement Learning is a library with tools to train transformer language models with Reinforcement Learning using different steps like  Supervised Fine-tuning step (SFT), Reward Modeling step (RM) to the Proximal Policy Optimization (PPO) step. 


Hugging Face SFT refers to the traditional approach of fine-tuning a pre-trained language model on a labeled dataset specific to a target task. This process involves continuing the training of the entire model on a smaller, task-specific dataset, allowing the model to adjust its weights to perform well on this new task. SFT is widely used for adapting general-purpose pre-trained models to specialized applications, from sentiment analysis and text classification to question-answering and beyond. It leverages the broad knowledge captured during pre-training, focusing it on the nuances of the specific task at hand. SFT is a cornerstone technique in transfer learning, enabling the rapid deployment of state-of-the-art models across a wide array of NLP tasks.

## Step 1 - Load Dataset

we're going to play around with this cool thing called the Guanaco dataset. It's basically a neat, tidy little piece of the bigger OpenAssistant dataset, and its tweaked just right for training chatbots that can do a bit of everything.

https://huggingface.co/datasets/timdettmers/openassistant-guanaco.

It consists of Human and Assistant messages in Conversational format. 

Sample format

`### Human: Can you explain abc

`### Assistant: abc is this... and that and so on..


In [2]:
from datasets import load_dataset

dataset_name = "timdettmers/openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Step 2 - Load the Base model

We will use the https://huggingface.co/TinyPixel/Llama-2-7B-bf16-sharded model.

We will quantize in 4 bit.

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

model-00001-of-00014.safetensors:   0%|          | 0.00/981M [00:00<?, ?B/s]

model-00002-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00003-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

model-00012-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00013-of-00014.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00014.safetensors:   0%|          | 0.00/847M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

## Step 3 - Load Tokenizer

Let's also load the tokenizer below

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

## Step 3 a - Run Inference on Un Tuned Model

Once we run inference on the Base/Untuned Model, we can see that it is just predicting the next words or sentences.
Once we input our prompts, It just gives similar words or sentences and what could be the next sentence.
These just look like simple sentence completions.

For example, for the prompt "How to make a iphone",
it comes back with responses like iphone 3gs wallpapers, iphone wallpapers, iphone. Iphone 6s plus.
Iphone 7. 48 iphone 6 plus wallpaper. 1600 x 1200 iphone 6 plus wallpaper, and so on.


For example, for the prompt "Give a Intro about Economy",
it comes back with responses like  Give a Intro about the economy of the United States, Give a Intro about the economy of Japan, Give a Intro about the economy of China, Give a Intro about the economy of India, and so on.

It is not doing any response in the conversational assistant format which is what we need it to do. This is the reason we will fine tune
the base model based on conversational data set. Once we fine tune, we will use the same prompts and see how the model produces responses 
in conversational assistant format. 

In [5]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)


input_query = """ How to make a iphone".
"""

sequences = pipeline(
   input_query,
    max_length=300,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result", seq)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Result {'generated_text': ' How to make a iphone".\n Unterscheidung der Typen von Anwendungen. 1280x1024 iphone 3gs wallpapers, iphone wallpapers, iphone. Iphone 6s plus.\nIphone 7. 48 iphone 6 plus wallpaper. 1600 x 1200 iphone 6 plus wallpaper.\nIphone 6 plus wallpaper 48 iphone 6 plus wallpaper.\n48 iPhone 6 Plus wallpaper. iPhone 6 Plus wallpaper.\n48 iPhone Plus. Iphone 6 Plus wallpaper.'}


In [6]:
for seq in sequences:
    print("Result", seq['generated_text'])

Result  How to make a iphone".
 Unterscheidung der Typen von Anwendungen. 1280x1024 iphone 3gs wallpapers, iphone wallpapers, iphone. Iphone 6s plus.
Iphone 7. 48 iphone 6 plus wallpaper. 1600 x 1200 iphone 6 plus wallpaper.
Iphone 6 plus wallpaper 48 iphone 6 plus wallpaper.
48 iPhone 6 Plus wallpaper. iPhone 6 Plus wallpaper.
48 iPhone Plus. Iphone 6 Plus wallpaper.


In [7]:
input_query = """ Give a Intro about Economy".
"""

sequences = pipeline(
   input_query,
    max_length=300,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result", seq)


Result {'generated_text': ' Give a Intro about Economy".\n nobody will read this post.\nI have been thinking of doing something similar to this for a long time. I have a lot of things I\'ve written and would love a place to put it all in one place so people can find it all and not have to look all over to find what they want to read.\nI\'ve also been looking for a place to put my writtings.\nI think it\'s a great idea.'}


In [8]:
for seq in sequences:
    print("Result", seq['generated_text'])

Result  Give a Intro about Economy".
 nobody will read this post.
I have been thinking of doing something similar to this for a long time. I have a lot of things I've written and would love a place to put it all in one place so people can find it all and not have to look all over to find what they want to read.
I've also been looking for a place to put my writtings.
I think it's a great idea.


In [9]:
input_query = """### Human: Give a Intro about Economy ".
Explain it to a kid.
  ### Assistant:
"""


sequences = pipeline(
   input_query,
    max_length=300,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print("Result", seq)

Result {'generated_text': '### Human: Give a Intro about Economy ".\nExplain it to a kid.\n  ### Assistant:\n  - #### Give a Intro about Economy\n  - #### Give a Intro about the economy of the United States\n  - #### Give a Intro about the economy of Japan\n  - #### Give a Intro about the economy of China\n  - #### Give a Intro about the economy of Russia\n  - #### Give a Intro about the economy of India\n  - #### Give a Intro about the economy of Germany\n  - #### Give a Intro about the economy of Brazil\n  - #### Give a Intro about the economy of Canada\n  - #### Give a Intro about the economy of Mexico\n  - #### Give a Intro about the economy of South Korea\n  - #### Give a Intro about the economy of France\n  - #### Give a Intro about the economy of Australia\n  - #### Give a Intro about the economy of the Philippines\n  - #### Give a Intro about the economy of the United Kingdom\n  - #### Give a Intro about the economy of Spain\n  - #### Give a Intro about the economy of Turkey\n\

In [10]:
for seq in sequences:
    print("Result", seq['generated_text'])

Result ### Human: Give a Intro about Economy ".
Explain it to a kid.
  ### Assistant:
  - #### Give a Intro about Economy
  - #### Give a Intro about the economy of the United States
  - #### Give a Intro about the economy of Japan
  - #### Give a Intro about the economy of China
  - #### Give a Intro about the economy of Russia
  - #### Give a Intro about the economy of India
  - #### Give a Intro about the economy of Germany
  - #### Give a Intro about the economy of Brazil
  - #### Give a Intro about the economy of Canada
  - #### Give a Intro about the economy of Mexico
  - #### Give a Intro about the economy of South Korea
  - #### Give a Intro about the economy of France
  - #### Give a Intro about the economy of Australia
  - #### Give a Intro about the economy of the Philippines
  - #### Give a Intro about the economy of the United Kingdom
  - #### Give a Intro about the economy of Spain
  - #### Give a Intro about the economy of Turkey

### Assistant:
- #### Give a Intro about

In [11]:
input_query = """ Convert below sentence to harry potter spell format - throw ball".
"""

sequences = pipeline(
   input_query,
    max_length=300,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result", seq)


Result {'generated_text': ' Convert below sentence to harry potter spell format - throw ball".\n everybody is doing it.\n I\'ve tried the code below. I get a blank screen.\nI\'ve looked at many examples, but none seem to work.\nI\'ve tried to use the "onclick" attribute to no avail.\n\nI need to be able to use the spell "throw-ball"\n\n\\begin{code}\n<button onclick="throwball()">throw ball</button>\n\nfunction throwball(){\n   var x=100;\n   var y=100;\n   throwball(x,y)\n  }\n\nfunction throwball(x,y){\n  document.getElementById(x).innerHTML=\'<p class="p" id="ball">ball\n\';\n  document.getElementById(y).innerHTML=\'<p class="p" id="ball">ball\n\';\n}\n\\end{code}\n\nAnswer: You need to pass the arguments in as a function call, like so:\n\n\\begin{code}\nfunction throwball(x, y) {\n  document.getElementById(x).innerHTML = \'<p class="p" id="ball">ball\n\';\n  document.getElementById(y).innerHTML = \'<p class="p" id="ball">ball\n\';\n}\n\n<button onclick="throwball()">throw'}


In [12]:
for seq in sequences:
    print("Result", seq['generated_text'])

Result  Convert below sentence to harry potter spell format - throw ball".
 everybody is doing it.
 I've tried the code below. I get a blank screen.
I've looked at many examples, but none seem to work.
I've tried to use the "onclick" attribute to no avail.

I need to be able to use the spell "throw-ball"

\begin{code}
<button onclick="throwball()">throw ball</button>

function throwball(){
   var x=100;
   var y=100;
   throwball(x,y)
  }

function throwball(x,y){
  document.getElementById(x).innerHTML='<p class="p" id="ball">ball
';
  document.getElementById(y).innerHTML='<p class="p" id="ball">ball
';
}
\end{code}

Answer: You need to pass the arguments in as a function call, like so:

\begin{code}
function throwball(x, y) {
  document.getElementById(x).innerHTML = '<p class="p" id="ball">ball
';
  document.getElementById(y).innerHTML = '<p class="p" id="ball">ball
';
}

<button onclick="throwball()">throw


## Step 4 - Set up Lora Config and PEFT config

We will load the configuration in order to create the LoRA model.  it is important to consider all linear layers in the transformer block for maximum performance. 

In [13]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj"]
)

## Step 5 - Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. 

In [14]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 4
optim = "paged_adamw_32bit"
save_steps = 10
logging_steps = 5
learning_rate = 2e-4
max_grad_norm = 0.3
#max_steps = 500
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
)

Then finally pass everthing to the trainer

In [15]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [16]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

## Step 6 - Train the model

We will train the model by calling `trainer.train()`

NOTE : When prompted for weights and balances api key, enter it. (we can sign up for wandb using google account for free)

In [ ]:
trainer.train()



![llama_training_loss](llama_training_loss.png)

![llama_training_loss](llama_training_loss_2.png)


During training, the model should converge 

## Step 6 a - Save the model

The `SFTTrainer` also takes care of properly saving only the adapters during training instead of saving the entire model. We can also publish the model to HF hub.
The model is saved in the cache memory by default. We can load it from output directory and create new tokenizer 
and run generate on the model. 
For our example, we will just use the pipeline and model from cache

## Step 7  - Run Inference on the Fine Tuned Model

Once we run inference on the Fine- tuned Model, we can see that it gives responses in the converstational assistant 
format which is what we need it to do.

For example, for the prompt "How to make a iphone",
it comes back with responses like "Answer: If you are using xcode 4.5 then use the simulators provided by the simulator itself. It is much faster than the simulators provided by Xcode".

Another Answer: You can use the simulator in Xcode to run your app on an iPhone or iPad.  If you have an iPhone, you can also download the iOS SDK and install the device simulator to run your application on that.

For example, for the prompt "Give a Intro about Economy",
it comes back with responses like  "Economy is the way we use the resources we have available, such as money, land, and labor, to produce things that we need and want for our society. It is a system of trade and exchange, where people and businesses buy and sell goods and services in order to meet their needs and desires. Economic systems are different from place to place and from time to time, but the goal is always to maximize our resources and use them in the most efficient way possible"

From the above examples, we can clearly see that fine tuned model has understanding of the conversational pattern we wanted it to learn.


In [18]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

input_query = """ How to make a iphone".
"""

sequences = pipeline(
   input_query,
    max_length=300,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print("Result", seq['generated_text'])

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Result  How to make a iphone".

Answer: If you are using xcode 4.5 then use the simulators provided by the simulator itself. It is much faster than the simulators provided by Xcode.

Answer: You can use the simulator in Xcode to run your app on an iPhone or iPad.  If you have an iPhone, you can also download the iOS SDK and install the device simulator to run your application on that.

If your goal is to have the app running on a real iPhone/iPad, you can also use the iPhone Configuration Utility, or the iTunes Device Tree. You should read the documentation on the device, to find out how to do this.

Answer: There's no way to get the real device to run your app.

You could use an [iPhone Emulator](http://www.iphoneemulator.com/) or a [Simulator](http://www.iphonesimulator.org/) to simulate a real iPhone.

Comment: You can get an iPad simulator for free from the app store and use it with the iPhone SDK.  I think that's the best option for a free solution.

Comment: @Jonathan: I don't th

In [19]:
for seq in sequences:
    print("Result", seq['generated_text'])

Result  How to make a iphone".

Answer: If you are using xcode 4.5 then use the simulators provided by the simulator itself. It is much faster than the simulators provided by Xcode.

Answer: You can use the simulator in Xcode to run your app on an iPhone or iPad.  If you have an iPhone, you can also download the iOS SDK and install the device simulator to run your application on that.

If your goal is to have the app running on a real iPhone/iPad, you can also use the iPhone Configuration Utility, or the iTunes Device Tree. You should read the documentation on the device, to find out how to do this.

Answer: There's no way to get the real device to run your app.

You could use an [iPhone Emulator](http://www.iphoneemulator.com/) or a [Simulator](http://www.iphonesimulator.org/) to simulate a real iPhone.

Comment: You can get an iPad simulator for free from the app store and use it with the iPhone SDK.  I think that's the best option for a free solution.

Comment: @Jonathan: I don't th

In [20]:
input_query = """ Give a Intro about Economy".
"""

sequences = pipeline(
   input_query,
    max_length=300,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print("Result", seq)

Result {'generated_text': ' Give a Intro about Economy".\n\nComment: @D.W. I have updated my answer with some more information. Hopefully that helps.\n\nAnswer: I would recommend using a [CSV file](http://www.csvsamples.eu/).  This will allow you to use your data as a table with headers.\n\nIf you are using Excel you can also create a [Pivot Table](http://office.microsoft.com/en-ca/excel-help/create-a-pivot-table-HP005203109.aspx).\n\nComment: Thanks! That\'s exactly what I was looking for!'}


In [21]:
for seq in sequences:
    print("Result", seq['generated_text'])

Result  Give a Intro about Economy".

Comment: @D.W. I have updated my answer with some more information. Hopefully that helps.

Answer: I would recommend using a [CSV file](http://www.csvsamples.eu/).  This will allow you to use your data as a table with headers.

If you are using Excel you can also create a [Pivot Table](http://office.microsoft.com/en-ca/excel-help/create-a-pivot-table-HP005203109.aspx).

Comment: Thanks! That's exactly what I was looking for!


In [22]:
input_query = """### Human: Give a Intro about Economy ".
Explain it to a kid.
  ### Assistant:
"""


sequences = pipeline(
   input_query,
    max_length=300,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print("Result", seq['generated_text'])

Result ### Human: Give a Intro about Economy ".
Explain it to a kid.
  ### Assistant:
  Economy is the way we use the resources we have available, such as money, land, and labor, to produce things that we need and want for our society. It is a system of trade and exchange, where people and businesses buy and sell goods and services in order to meet their needs and desires. Economic systems are different from place to place and from time to time, but the goal is always to maximize our resources and use them in the most efficient way possible.

Economy plays a vital role in our lives and is essential for our society to function and grow. Understanding how the economy works, and how it affects our daily lives, is important for making informed decisions about our own lives and our future.
  ### Human: What is a currency?
  ### Assistant: A currency is a medium of exchange for goods and services. It can be used as a form of payment or for storing value. Currencies are typically issued by go

In [23]:
input_query = """ Convert below sentence to harry potter spell format - throw ball".
"""

sequences = pipeline(
   input_query,
    max_length=300,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print("Result", seq['generated_text'])


Result  Convert below sentence to harry potter spell format - throw ball".

I am looking for a function that takes a word and return it with a spell format.

\begin{code}
function getSpell(word) {
   return word + " " + " " + "Throw " + " " + "Ball"
}
\end{code}

I want to know if there is a function in javascript that will convert the word to a harry potter spell format.

Comment: I'm not sure what you mean by "a function"... do you mean a function in JavaScript?

Comment: I am not looking for a library that does it, I am looking for a function in javascript.

Answer: You can use `String.prototype.replace`:

\begin{code}
function getSpell(word) {
    return word.replace(/([a-z])/g, function (letter){
      return letter.toUpperCase();
    });
  }

  alert(getSpell('Throw Ball'));
  alert(getSpell('Throw ball'));
  alert(getSpell('throw Ball'));
  alert(getSpell('throw ball'));
  alert(getSpell('throwBall'));
  alert(getSpell('throwBall'));
 
